In [ ]:
import pandas as pd
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from dotenv import load_dotenv
load_dotenv()

In [ ]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.1
)

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

class RouteDecision(BaseModel):
    """
    Determines which specialized agent should handle the user query.
    """

    route: Literal["math", "code", "general"] = Field(
        ...,
        description="Select 'math' for calculations, 'code' for programming help, or 'general' for other questions."
    )

    reasoning: str = Field(
        ...,
        description="Short explanation of why this route was chosen."
    )


### Router Node

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

router_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a routing agent. Decide the correct agent."),
    ("human", "{question}")
])

structured_router = router_prompt | llm.with_structured_output(RouteDecision)

@traceable(name="Router Node")
def router_node(state):
    question = state["question"]
    decision = structured_router.invoke({"question": question})
    
    return {
        "route": decision.route,
        "reasoning": decision.reasoning
    }


In [ ]:
@traceable(name="Math Node")
def math_agent(state):
    question = state["question"]
    response = llm.invoke(f"Solve the math problem step by step: {question}")
    return {"answer": response.content}


In [ ]:
@traceable(name="Code Node")
def code_agent(state):
    question = state["question"]
    response = llm.invoke(f"Provide programming help for: {question}")
    return {"answer": response.content}


In [ ]:
@traceable(name="General Node")
def general_agent(state):
    question = state["question"]
    response = llm.invoke(f"Answer the general knowledge question: {question}")
    return {"answer": response.content}


In [ ]:

from typing import TypedDict

class GraphState(TypedDict):
    question: str
    route: str
    reasoning: str
    answer: str



In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)

# Add nodes
graph.add_node("router", router_node)
graph.add_node("math_agent", math_agent)
graph.add_node("code_agent", code_agent)
graph.add_node("general_agent", general_agent)

# Routing logic
def route_condition(state):
    return state["route"]

graph.add_conditional_edges(
    "router",
    route_condition,
    {
        "math": "math_agent",
        "code": "code_agent",
        "general": "general_agent"
    }
)

# End after each agent
graph.add_edge("math_agent", END)
graph.add_edge("code_agent", END)
graph.add_edge("general_agent", END)

graph.set_entry_point("router")

app = graph.compile()


In [ ]:
result = app.invoke({"question": "Write a Python function to reverse a string"})

In [ ]:
print("Route Chosen:", result["route"])

In [ ]:
print("Reasoning:", result["reasoning"])

In [ ]:
print("Answer:\n", result["answer"])